In [13]:
import pandas as pd
import os

In [14]:
school_data_to_load = os.path.join('Resources','schools_complete.csv')
student_data_to_load = os.path.join('Resources','students_complete.csv')

school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)



pycity_schools = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
pycity_schools

#Check Data
#pycity_schools.count()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


# District Summary

In [15]:
#Total Schools, Students, and Budget
count_schools = school_data["school_name"].value_counts()
total_schools = count_schools.sum()

count_students = student_data["student_name"].value_counts()
total_students = count_students.sum()

total_budget = school_data["budget"].sum()
total_budget

#Average scores
avg_math = pycity_schools["math_score"].mean()

avg_reading = pycity_schools["reading_score"].mean()

#Percent passing 
pass_math = pycity_schools.loc[pycity_schools["math_score"] >= 70]
pct_pass_math = (pass_math.shape[0]/total_students)*100

pass_reading = pycity_schools.loc[pycity_schools["reading_score"] >= 70]
pct_pass_reading = (pass_reading.shape[0]/total_students)*100

pass_all = pycity_schools.loc[(pycity_schools["math_score"] >= 70) & (pycity_schools["reading_score"] >= 70)]
pct_pass_all = (pass_all.shape[0]/total_students)*100

In [16]:
#Put summary data into dictionary 
district_dicts = [{"Total Schools":total_schools,
                  "Total Students":total_students,
                  "Total Budget":total_budget, 
                  "Average Math Score":avg_math,
                  "Average Reading Score":avg_reading,
                  "% Passing Math":pct_pass_math,
                  "% Passing Reading":pct_pass_reading,
                  "% Overall Passing":pct_pass_all}]

#Create Data Frame for District Summary data
district_summary_df = pd.DataFrame(district_dicts)

#Format Columns in District Summary
district_summary_df["Total Students"] = district_summary_df["Total Students"].astype(float).map("{:,.0f}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].astype(float).map("${:,.2f}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].astype(float).map("{:.2f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Math Score"].astype(float).map("{:.2f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].astype(float).map("{:.2f}%".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].astype(float).map("{:.2f}%".format)
district_summary_df[ "% Overall Passing"] = district_summary_df[ "% Overall Passing"].astype(float).map("{:.2f}%".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,78.99,74.98%,85.81%,65.17%


# School Summary 

In [5]:
#Set Index to School Name
school_summary_df = school_data.set_index("school_name").drop(columns = "School ID").sort_index()

#Create Column and Variable for Per Student Budget
school_summary_df["Per Student Budget"] = school_summary_df["budget"]/school_summary_df["size"]

#Find Score Data for each school using .groupby
school_groupby = pycity_schools.groupby(["school_name"])

school_summary_df["Average Math Score"] = school_groupby["math_score"].mean()
school_summary_df["Average Reading Score"] = school_groupby["reading_score"].mean()

school_summary_df["% Passing Math"] = (pass_math.groupby("school_name").count()["Student ID"]/school_summary_df["size"])*100
school_summary_df["% Passing Reading"] = (pass_reading.groupby("school_name").count()["Student ID"]/school_summary_df["size"])*100
school_summary_df["% Overall Passing"] = (pass_all.groupby("school_name").count()["Student ID"]/school_summary_df["size"])*100

school_summary_df 

,type,size,budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


# Top Performing Schools (By % Overall Passing)

In [6]:
#Sort the Schools from Highest Passing to Lowest, then display the top five schools
top_five_schools = school_summary_df.sort_values("% Overall Passing", ascending=False)
top_five_schools.head(5)

,type,size,budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


# Bottom Performing Schools (By % Overall Passing)

In [7]:
#Sort the schools from Lowest Passing to Highest, then display the bottom five schools
bottom_five_schools = school_summary_df.sort_values("% Overall Passing")
bottom_five_schools.head(5)

,type,size,budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


# Math Scores by Grade

In [8]:
#Find the average math scores for each grade
math_gr9=student_data.loc[student_data["grade"] == "9th"].groupby("school_name")["math_score"].mean()
math_gr10=student_data.loc[student_data["grade"] == "10th"].groupby("school_name")["math_score"].mean()
math_gr11=student_data.loc[student_data["grade"] == "11th"].groupby("school_name")["math_score"].mean()
math_gr12=student_data.loc[student_data["grade"] == "12th"].groupby("school_name")["math_score"].mean()

#Put the averages into a Data Frame
math_by_grade_df = pd.DataFrame({"9th":math_gr9,"10th":math_gr10,"11th":math_gr11,"12th":math_gr12})
math_by_grade_df.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164


# Reading Scores by Grade

In [9]:
#Find the average reading scores for each grade
reading_gr9=student_data.loc[student_data["grade"] == "9th"].groupby("school_name")["reading_score"].mean()
reading_gr10=student_data.loc[student_data["grade"] == "10th"].groupby("school_name")["reading_score"].mean()
reading_gr11=student_data.loc[student_data["grade"] == "11th"].groupby("school_name")["reading_score"].mean()
reading_gr12=student_data.loc[student_data["grade"] == "12th"].groupby("school_name")["reading_score"].mean()

#Put the averages into a Data Frame
reading_by_grade_df = pd.DataFrame({"9th":reading_gr9,"10th":reading_gr10,"11th":reading_gr11,"12th":reading_gr12})
reading_by_grade_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


# Scores by School Spending

In [10]:
#Create bins and labels for the groups based on Per Student Budget Data in School Summary
spending_bins = [0, 584, 629, 644, 675]
spending_labels = ["<$584","$585-629","$630-644","$645-675"]

#Group schools by spending
school_summary_df["Spending Ranges (Per Student)"] = pd.cut(school_summary_df["Per Student Budget"], spending_bins, labels=spending_labels)
by_spending_df = school_summary_df.groupby(["Spending Ranges (Per Student)"])

#Find the average outcomes for schools in each spending group
math_by_spending = by_spending_df["Average Math Score"].mean()
reading_by_spending = by_spending_df["Average Reading Score"].mean()
pass_math_by_spending = by_spending_df["% Passing Math"].mean()
pass_reading_by_spending = by_spending_df["% Passing Reading"].mean()
pass_all_by_spending = by_spending_df["% Overall Passing"].mean()

#Put the averages into a Data Frame
scores_by_spending = pd.DataFrame({"Average Math Score":math_by_spending,
                                   "Average Reading Score":reading_by_spending,
                                   "% Passing Math":pass_math_by_spending,
                                   "% Passing Reading":pass_reading_by_spending,
                                   "% Overall Passing":pass_all_by_spending})
scores_by_spending.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.455399,83.933814,93.460096,96.610877,90.369459
$585-629,81.899826,83.155286,87.133538,92.718205,81.418596
$630-644,78.518855,81.624473,73.484209,84.391793,62.857656
$645-675,76.997210,81.027843,66.164813,81.133951,53.526855


# Scores by School Size

In [11]:
#Create bins and labels for the groups based on size in School Summary
size_bins = [0, 999, 1999, 5000]
size_labels = ["<1000","1000-1999","2000-5000"]

#Group Schools by size
school_summary_df["School Size"] = pd.cut(school_summary_df["size"], size_bins, labels=size_labels)
by_size_df = school_summary_df.groupby(["School Size"])

#Find the average outcomes for schools in each size group
math_by_size = by_size_df["Average Math Score"].mean()
reading_by_size = by_size_df["Average Reading Score"].mean()
pass_math_by_size = by_size_df["% Passing Math"].mean()
pass_reading_by_size = by_size_df["% Passing Reading"].mean()
pass_all_by_size = by_size_df["% Overall Passing"].mean()

#Put the averages into a Data Frame
scores_by_size = pd.DataFrame({"Average Math Score":math_by_size,
                                   "Average Reading Score":reading_by_size,
                                   "% Passing Math":pass_math_by_size,
                                   "% Passing Reading":pass_reading_by_size,
                                   "% Overall Passing":pass_all_by_size})
scores_by_size.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
<1000,83.821598,83.929843,93.550225,96.099437,89.883853
1000-1999,83.374684,83.864438,93.599695,96.790680,90.621535
2000-5000,77.746417,81.344493,69.963361,82.766634,58.286003


# Scores by School Type

In [12]:
#Group schools by school type
by_type_df = school_summary_df.groupby(["type"])

#Find average outcomes by type
math_by_type = by_type_df["Average Math Score"].mean()
reading_by_type = by_type_df["Average Reading Score"].mean()
pass_math_by_type = by_type_df["% Passing Math"].mean()
pass_reading_by_type = by_type_df["% Passing Reading"].mean()
pass_all_by_type = by_type_df["% Overall Passing"].mean()

#Put averages into a DataFrame
scores_by_type = pd.DataFrame({"Average Math Score":math_by_type,
                                   "Average Reading Score":reading_by_type,
                                   "% Passing Math":pass_math_by_type,
                                   "% Passing Reading":pass_reading_by_type,
                                   "% Overall Passing":pass_all_by_type})
scores_by_type.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


# Analysis

Using the tables created above, several trends are evident for PyCity Schools:

(1) Differences in math scores are significantly more exaggerated than reading scores across the different schools. Whether the schools are evaluated by spending, size, and type, math education is more sensitive than reading. 

(2) Students have higher scores in both subjects in smaller, better-funded schools. 

(3) When schools are sorted by type (District and Charter), the scores for charter schools are higher. Charter schools in this data set to be smaller than the district schools, and it has already been noted that smaller sizes appear to lead to higher scores. At the same time, however, the bottom five performing schools--which are all district schools--outspent the top five performing schools--all charter. 